## Assigment: Neural network basics

Soft deadline: 16.09.18 at 23.59

Hard deadline: 18.09.18 at 23.59

For this task I intentionally provide no boilerplate code, because very puprpose of this task is getting you comforatable with basic code template for desiging NNs in pytorch. I higly recommend you to revisit all the last seminar materials.

#### Task

* Implement simple **fully-convolutional** neural architecture for classification. Make sure it is small enought to run on your home machine.
* Provide dataset visulization.
* Provide train/test split and validation

#### Requirements

* Architecture should derive from `torch.nn.Module`
* Use `torch.utils.data.Dataset` and `torch.utils.data.DataLoader`. But if you manage co simplify this step using dataset `torchivision`, I will only encourage you.
* Implement at least one data transformer, but make sure it is useful for classification task.
* Use FashionMNIST dataset https://github.com/zalandoresearch/fashion-mnist
* Make sure you can fix random seed for all components of your code to make experiments reproducible
* Since you architecure should be fully-convolutional, make sure it does not depend on input size.

In [123]:
%matplotlib inline
import matplotlib.pyplot as plt

import torch
import torchvision
import torchvision.transforms as transforms
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable
import torch.nn.functional as F
import random
from IPython import display
from sklearn import datasets, preprocessing

In [124]:
BATCH_SIZE = 4

transform = transforms.Compose([transforms.ToTensor(), transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
        #      or something else...

train_set = torchvision.datasets.FashionMNIST("./", train=True, transform=transform, download=True)
test_set = torchvision.datasets.FashionMNIST("./", train=False, transform=transform, download=True)
print(train_set.__len__)
print(test_set.__len__)

train_loader = torch.utils.data.DataLoader(train_set, batch_size=BATCH_SIZE)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=BATCH_SIZE)
# fst_batch = next(iter(train_loader))

# print(fst_batch[0].size(), fst_batch[1].size())
# print(fst_batch[0][0])
    
    

<bound method MNIST.__len__ of Dataset FashionMNIST
    Number of datapoints: 60000
    Split: train
    Root Location: ./
    Transforms (if any): Compose(
                             ToTensor()
                             Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                         )
    Target Transforms (if any): None>
<bound method MNIST.__len__ of Dataset FashionMNIST
    Number of datapoints: 10000
    Split: test
    Root Location: ./
    Transforms (if any): Compose(
                             ToTensor()
                             Normalize(mean=(0.5, 0.5, 0.5), std=(0.5, 0.5, 0.5))
                         )
    Target Transforms (if any): None>


In [125]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(1, 6, 3, padding=1)
        self.pool = nn.MaxPool2d(2, 2)
        self.conv2 = nn.Conv2d(6, 14, 3, padding=1)
        self.fc1 = nn.Linear(14 * 7 * 7, 100)
        self.fc2 = nn.Linear(100, 10)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = self.pool(F.relu(self.conv2(x)))
        x = x.view(-1, 14 * 7 * 7)
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        return x

net = Net()
print(net)

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

for epoch in range(3):
    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):
        inputs, labels = data
        inputs, labels = Variable(inputs), Variable(labels)

        optimizer.zero_grad()

        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()
        if i % 100 == 0:
            print(f"Loss {loss.item()}")

print('Finished Training')


Net(
  (conv1): Conv2d(1, 6, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 14, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (fc1): Linear(in_features=686, out_features=100, bias=True)
  (fc2): Linear(in_features=100, out_features=10, bias=True)
)
Loss 2.318204879760742
Loss 2.2634143829345703
Loss 2.248753070831299
Loss 2.2927303314208984
Loss 2.3740806579589844
Loss 1.7790218591690063
Loss 2.1941823959350586
Loss 1.5688652992248535
Loss 1.810684084892273
Loss 1.511436939239502
Loss 1.4796191453933716
Loss 1.268081784248352
Loss 0.7853357195854187
Loss 1.6493446826934814
Loss 0.9863147735595703
Loss 1.175879716873169
Loss 1.117272138595581
Loss 0.8955055475234985
Loss 2.2493650913238525
Loss 2.6554465293884277
Loss 0.3419641852378845
Loss 0.21923577785491943
Loss 1.1609759330749512
Loss 2.7905001640319824
Loss 2.16614031791687
Loss 0.7399162650108337
Loss 1.14633464813

Loss 0.1265583038330078
Loss 0.8398325443267822
Loss 0.17078137397766113
Loss 0.17193222045898438
Loss 0.14173710346221924
Loss 0.2862095832824707
Loss 0.01935434341430664
Loss 0.12591075897216797
Loss 0.3139769434928894
Loss 0.938319981098175
Loss 0.6501681804656982
Loss 0.18384861946105957
Loss 0.027099132537841797
Loss 0.019523143768310547
Loss 0.1345679759979248
Loss 0.13850951194763184
Loss 0.007781028747558594
Loss 2.735799789428711
Loss 0.06613945960998535
Loss 0.21978628635406494
Loss 0.3903985023498535
Loss 0.09509682655334473
Loss 1.1852006912231445
Loss 0.40878140926361084
Loss 0.495137095451355
Loss 0.01191401481628418
Loss 0.028983592987060547
Loss 0.017557859420776367
Loss 0.2632012367248535
Loss 0.48706793785095215
Loss 0.20905566215515137
Loss 0.40713930130004883
Loss 0.2995946407318115
Loss 0.14803576469421387
Loss 0.09211957454681396
Loss 0.3123129606246948
Loss 0.35259056091308594
Loss 0.31314969062805176
Loss 0.052950382232666016
Loss 0.5780520439147949
Loss 0.06328

In [126]:
correct = 0
total = 0
for data in test_loader:
    images, labels = data
    outputs = net(Variable(images))
    _, predicted = torch.max(outputs.data, 1)   # Find the class index with the maximum value.
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 88 %
